In [1]:
import sys
sys.path.append('../')

In [2]:
import fiftyone as fo
from fiftyone import ViewField as F
from tracking import TrackingMetrics
from tracking.utils import compute_metrics_by_sequence, results_to_df, get_sequence_info, classify_num_objects

In [3]:
dataset = fo.load_dataset("SENTRY_VIDEOS_DATASET_QA")
models = [
    dict(name="yellow-davit-21_09_2023_N_LN1_ep294_TRACKER", conf_thr=0.0),
]

In [6]:
det_gt_field = "ground_truth_det"

metric_parameters = {'non-existing-parameter': 0.2 , 'max_iou': 1.0, 'metrics': ['num_frames', 'num_switches', 'num_matches', 'mota', 'motp', 'idf1', 'idp', 'idr', 'num_false_positives',
                        'num_misses', 'num_fragmentations', 'precision', 'recall', 'num_unique_objects']}

sequence_list = ['PROACT_CELADON_S@6m_FB_SUNRISE_20230720_2023_02_07_17_31_58',
                'PROACT_CELADON_S@6m_FB_SUNRISE_20230720_2023_02_07_17_51_10',
                'PROACT_CELADON_S@6m_FB_SUNRISE_20230720_2023_02_07_18_03_24',
                'Sentry_2022_08_Portugal_2022_08_16_11_23_32',
                'Sentry_2022_08_Portugal_2022_08_16_16_10_15',
                'Sentry_2022_08_Portugal_2022_08_29_20_30_47',
                'Sentry_2022_08_Portugal_2022_08_29_20_32_19',
                'Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_11_26_53',
                'Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_14_44_21',
                'Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_15_02_32',
                'Sentry_2023_02_08_PROACT_CELADON_@6m_MOB_2023_02_08_12_32_23',
                'Sentry_2023_05_France_FB_WL_2023_02_07_16_17_26',
                'Sentry_2023_05_France_FB_WL_2023_02_07_16_22_33',
                'Sentry_2023_07_Portugal_FB_BS_2023_07_13_12_45_38',
                'Sentry_2023_08_Portugal_FB_BS_2023_08_11_16_07_11',
                'Sentry_2023_08_Portugal_FB_BS_2023_08_11_17_00_51']


for model in models:
        model['metrics'] = compute_metrics_by_sequence(view=dataset.select_group_slices(["thermal_wide"]) \
                                                    .match_tags("LONG_VIDEOS", False) \
                                                     #if Long videos tag is ommited, fiftyone crashes due to a MongoDB error
                                                    .match_tags("TRACKER_TEST", True) \
                                                    #sequences marked with TRACKER_TEST tag have track id on the ground truth
                                                    .filter_labels(f"frames.{model['name']}", F("confidence") >= model['conf_thr'], only_matches=False),
                                                gt_field=det_gt_field,
                                                pred_field=model['name'],
                                                metric_fn=TrackingMetrics,
                                                metric_kwargs=metric_parameters,
                                                sequence_list=None) #if sequence_list is None the metrics will be computed for every sequence

Failed to compute metrics for sequence PROACT_CELADON_S@6m_FB_SUNRISE_20230720_2023_02_07_18_03_24
GT: 423
Pred: 0
Failed to compute metrics for sequence Sentry_2022_11_PROACT_CELADON_7.5M_MOB_2022_11_25_14_44_21
GT: 102
Pred: 0
Failed to compute metrics for sequence Sentry_2023_05_France_FB_WL_2023_02_07_16_17_26
GT: 0
Pred: 93
Failed to compute metrics for sequence Sentry_2023_05_France_FB_WL_2023_02_07_16_22_33
GT: 0
Pred: 72
Failed to compute metrics for sequence Sentry_2023_08_Portugal_FB_BS_2023_08_11_16_07_11
GT: 0
Pred: 0


In [8]:
df = results_to_df(metrics=models[0]['metrics'])

In [9]:
df

,num_frames,num_switches,num_matches,mota,motp,idf1,idp,idr,num_false_positives,num_misses,num_fragmentations,precision,recall,num_unique_objects,sequence
0,1289,22,1094,23.499268,87.848333,0.579416,0.498678,0.690337,773,250,17,0.590789,0.816984,4,PROACT_CELADON_S@6m_FB_SUNRISE_20230720_2023_0...
0,1074,39,2314,70.986531,56.340705,0.655564,0.659183,0.651984,364,394,48,0.866029,0.856571,5,PROACT_CELADON_S@6m_FB_SUNRISE_20230720_2023_0...
0,383,4,738,49.630128,41.740348,0.561687,0.843666,0.420982,0,745,28,1.000000,0.498991,5,Sentry_2021_11_Greece_2021_11_12_07_57_06
0,662,77,3720,77.570093,70.856950,0.732066,0.808763,0.666525,68,911,138,0.982406,0.806500,16,Sentry_2021_11_Greece_2021_11_12_12_20_26
0,350,11,410,41.019108,66.984203,0.442040,0.561886,0.364331,88,364,19,0.827112,0.536306,5,Sentry_2022_08_Portugal_2022_08_16_11_14_38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,920,18,1428,63.169643,79.816157,0.672614,0.852639,0.555357,13,794,75,0.991090,0.645536,4,Sentry_2023_05_Portugal_2023_05_25_17_09_36
0,2394,43,2922,51.629472,83.416031,0.198190,0.287349,0.151258,7,2681,152,0.997645,0.525151,8,Sentry_2023_05_Portugal_2023_05_26_17_48_26
0,1707,31,1737,65.617805,72.920447,0.553056,0.675361,0.466999,27,838,59,0.984958,0.678434,7,Sentry_2023_07_Portugal_FB_BS_2023_07_13_12_45_38
0,1503,18,463,30.819367,83.563097,0.339063,0.465041,0.266294,132,593,68,0.784666,0.447858,2,Sentry_2023_08_Portugal_FB_BS_2023_08_11_16_50_55
